In [2]:
import xlrd
import os
from pandas import Series,DataFrame
import numpy as np
import pandas as pd
import datetime

#### 1、通过header=None的方式读取excel

In [5]:
file_name='data/convert_2.xlsx'
read_data=pd.read_excel(file_name,sheetname=0,header=None)

In [6]:
read_data.head()

,0,1,2,3,4,5,6
0,年 月 日,风暴潮组,海浪组,海浪组,海浪组,海冰组,海冰组
1,NaN,主班,主班,副班,警报班,主班,主班
2,2017-12-01 00:00:00,刘秋兴,韦锋余,徐瑞,王娟娟,唐茂宁,刘琼
3,2017-12-02 00:00:00,刘秋兴,邢闯,韦锋余,徐瑞,唐茂宁,刘琼
4,2017-12-03 00:00:00,刘秋兴,侯放,邢闯,韦锋余,唐茂宁,刘琼


In [7]:
test_data=pd.DataFrame(np.arange(6).reshape(2,3))
test_data

,0,1,2
0,0,1,2
1,3,4,5


* 读取行标题

In [8]:
read_data.columns.tolist()

[0, 1, 2, 3, 4, 5, 6]

#### 1.1 去掉第一列，取剩下的前两行

In [9]:
# 会出错 KeyError: 1
# read_data[1]
read_data[:2]

,0,1,2,3,4,5,6
0,年 月 日,风暴潮组,海浪组,海浪组,海浪组,海冰组,海冰组
1,NaN,主班,主班,副班,警报班,主班,主班


#### 1.2获取第一列起始至结束；第0行起始至第一行的数据

In [10]:
columns_names=read_data.iloc[0:2,1:]
columns_names

,1,2,3,4,5,6
0,风暴潮组,海浪组,海浪组,海浪组,海冰组,海冰组
1,主班,主班,副班,警报班,主班,主班


In [11]:
columns_names.shape[1]

6

In [12]:
np.arange(columns_names.shape[1])

array([0, 1, 2, 3, 4, 5])

#### 1.3 修改columns

In [27]:
columns_names.columns=np.arange(columns_names.shape[1])

In [28]:
columns_names

,0,1,2,3,4,5
0,风暴潮组,海浪组,海浪组,海浪组,海冰组,海冰组
1,主班,主班,副班,警报班,主班,主班


#### 在组名上面在加入一行室名

In [29]:
columns_department=pd.DataFrame(columns=columns_names.columns,index=[2])
columns_department

,0,1,2,3,4,5
2,NaN,NaN,NaN,NaN,NaN,NaN


* 将columns_names与columns_department合并

In [30]:
columns_newmerage=pd.merge(columns_names,columns_department,how='outer')
columns_newmerage

,0,1,2,3,4,5
0,风暴潮组,海浪组,海浪组,海浪组,海冰组,海冰组
1,主班,主班,副班,警报班,主班,主班
2,NaN,NaN,NaN,NaN,NaN,NaN


* 调整位置，将2前移至0处  
不要使用reindex，直接通过为index赋值即可

In [31]:
columns_newmerage.reindex([1,2,0])

,0,1,2,3,4,5
1,主班,主班,副班,警报班,主班,主班
2,NaN,NaN,NaN,NaN,NaN,NaN
0,风暴潮组,海浪组,海浪组,海浪组,海冰组,海冰组


In [33]:
columns_newmerage.index=[1,2,0]
columns_newmerage

,0,1,2,3,4,5
1,风暴潮组,海浪组,海浪组,海浪组,海冰组,海冰组
2,主班,主班,副班,警报班,主班,主班
0,NaN,NaN,NaN,NaN,NaN,NaN


* 重新排序  
使用sort_index会重新返回一个新的df，而不是在原有的df基础上进行修改

In [36]:
columns_newmerage=columns_newmerage.sort_index()
columns_newmerage

,0,1,2,3,4,5
0,NaN,NaN,NaN,NaN,NaN,NaN
1,风暴潮组,海浪组,海浪组,海浪组,海冰组,海冰组
2,主班,主班,副班,警报班,主班,主班


* Nan不能通过==np.nan的方式判断是否为空
* 只能通过is np.nan的方式

#### 1.4为所有的nan值填充默认值（此处为指定一个值），以后改为文件名中的室名称

In [40]:
nan_name='预警室'
columns_newmerage=columns_newmerage.fillna(nan_name)
columns_newmerage

,0,1,2,3,4,5
0,预警室,预警室,预警室,预警室,预警室,预警室
1,风暴潮组,海浪组,海浪组,海浪组,海冰组,海冰组
2,主班,主班,副班,警报班,主班,主班


#### 3 获取人员主体dataframe

In [49]:
read_data[2:-1].head()

,0,1,2,3,4,5,6
2,2017-12-01 00:00:00,刘秋兴,韦锋余,徐瑞,王娟娟,唐茂宁,刘琼
3,2017-12-02 00:00:00,刘秋兴,邢闯,韦锋余,徐瑞,唐茂宁,刘琼
4,2017-12-03 00:00:00,刘秋兴,侯放,邢闯,韦锋余,唐茂宁,刘琼
5,2017-12-04 00:00:00,梁森栋,高志一,侯放,邢闯,唐茂宁,刘琼
6,2017-12-05 00:00:00,梁森栋,王久珂,高志一,侯放,唐茂宁,刘琼


#### 3.1 获取时间索引

In [50]:
read_data[2:-1][0].head()

2    2017-12-01 00:00:00
3    2017-12-02 00:00:00
4    2017-12-03 00:00:00
5    2017-12-04 00:00:00
6    2017-12-05 00:00:00
Name: 0, dtype: object

#### 3.2 重新设置第一列为索引  
使用df..set_index('列名')

In [51]:
convert_data=read_data[2:-1].set_index(0)

In [26]:
convert_data.head()

,1,2,3,4,5,6
0,,,,,,
2017-12-01,刘秋兴,韦锋余,徐瑞,王娟娟,唐茂宁,刘琼
2017-12-02,刘秋兴,邢闯,韦锋余,徐瑞,唐茂宁,刘琼
2017-12-03,刘秋兴,侯放,邢闯,韦锋余,唐茂宁,刘琼
2017-12-04,梁森栋,高志一,侯放,邢闯,唐茂宁,刘琼
2017-12-05,梁森栋,王久珂,高志一,侯放,唐茂宁,刘琼


* 注意使用set_index会默认将原来的列名（eg：0）作为新的索引的name  
所以若想删除0这一行，使用drop的方式不可行，会报错

In [27]:
convert_data.drop(0)

ValueError: labels [0] not contained in axis

In [28]:
convert_data.index

DatetimeIndex(['2017-12-01', '2017-12-02', '2017-12-03', '2017-12-04',
               '2017-12-05', '2017-12-06', '2017-12-07', '2017-12-08',
               '2017-12-09', '2017-12-10', '2017-12-11', '2017-12-12',
               '2017-12-13', '2017-12-14', '2017-12-15', '2017-12-16',
               '2017-12-17', '2017-12-18', '2017-12-19', '2017-12-20',
               '2017-12-21', '2017-12-22', '2017-12-23', '2017-12-24',
               '2017-12-25', '2017-12-26', '2017-12-27', '2017-12-28',
               '2017-12-29', '2017-12-30'],
              dtype='datetime64[ns]', name=0, freq=None)

* 但这个name貌似对检索并无影响

#### 3.3 根据时间索引取值（测试用）

In [29]:
convert_data[:1].shape

(1, 6)

In [30]:
convert_data[:1]

,1,2,3,4,5,6
0,,,,,,
2017-12-01,刘秋兴,韦锋余,徐瑞,王娟娟,唐茂宁,刘琼


In [31]:
datetime.datetime(2017,12,1)

datetime.datetime(2017, 12, 1, 0, 0)

* 注意从dataframe中取出一行就是一个series

In [32]:
type(convert_data.loc[datetime.datetime(2017,12,1)])

pandas.core.series.Series

In [33]:
convert_data.loc[datetime.datetime(2017,12,1)]

1    刘秋兴
2    韦锋余
3     徐瑞
4    王娟娟
5    唐茂宁
6     刘琼
Name: 2017-12-01 00:00:00, dtype: object

#### 3.4 尝试去掉索引列的name 0（测试用）

In [67]:
#convert_data[0].tolist()

### 4、重新赋值columns并通过层次化的方式重新读取数据

In [41]:
columns_newmerage

,0,1,2,3,4,5
0,预警室,预警室,预警室,预警室,预警室,预警室
1,风暴潮组,海浪组,海浪组,海浪组,海冰组,海冰组
2,主班,主班,副班,警报班,主班,主班


* 若直接将columns_names（是一个dataframe）  
直接赋值给要修改的dataframe的columns会出错：  
ValueError: Length mismatch: Expected axis has 6 elements, new values have 2 elements

In [44]:
columns_newmerage.iloc[0:1]

,0,1,2,3,4,5
0,预警室,预警室,预警室,预警室,预警室,预警室


In [45]:
columns_newmerage.loc[0]

0    预警室
1    预警室
2    预警室
3    预警室
4    预警室
5    预警室
Name: 0, dtype: object

In [46]:
columns_newmerage.loc[0].tolist()

['预警室', '预警室', '预警室', '预警室', '预警室', '预警室']

* 注意若直接将series作为列的话，会在列的前面加一个数字  
解决办法：将series.tolist()

In [47]:
#convert_data.columns=[columns_names.loc[0],columns_names.loc[1]]

In [53]:
columns_newmerage

,0,1,2,3,4,5
0,预警室,预警室,预警室,预警室,预警室,预警室
1,风暴潮组,海浪组,海浪组,海浪组,海冰组,海冰组
2,主班,主班,副班,警报班,主班,主班


In [57]:
convert_data.columns=[columns_newmerage.loc[0].tolist(),columns_newmerage.loc[1].tolist(),columns_newmerage.loc[2].tolist()]

In [58]:
convert_data.head()

预警室                        
           风暴潮组  海浪组            海冰组    
             主班   主班   副班  警报班   主班  主班
0                                      
2017-12-01  刘秋兴  韦锋余   徐瑞  王娟娟  唐茂宁  刘琼
2017-12-02  刘秋兴   邢闯  韦锋余   徐瑞  唐茂宁  刘琼
2017-12-03  刘秋兴   侯放   邢闯  韦锋余  唐茂宁  刘琼
2017-12-04  梁森栋  高志一   侯放   邢闯  唐茂宁  刘琼
2017-12-05  梁森栋  王久珂  高志一   侯放  唐茂宁  刘琼

In [59]:
convert_data['风暴潮组'].head()

,主班
0,
2017-12-01,刘秋兴
2017-12-02,刘秋兴
2017-12-03,刘秋兴
2017-12-04,梁森栋
2017-12-05,梁森栋


In [60]:
convert_data.iloc[0]

风暴潮组  主班     刘秋兴
海浪组   主班     韦锋余
      副班      徐瑞
      警报班    王娟娟
海冰组   主班     唐茂宁
      主班      刘琼
Name: 2017-12-01 00:00:00, dtype: object

In [61]:
convert_data.loc[datetime.datetime(2017,12,1)]

风暴潮组  主班     刘秋兴
海浪组   主班     韦锋余
      副班      徐瑞
      警报班    王娟娟
海冰组   主班     唐茂宁
      主班      刘琼
Name: 2017-12-01 00:00:00, dtype: object

* 会出错：  
TypeError: cannot do index indexing on <class 'pandas.tseries.index.DatetimeIndex'> with these indexers [1] of <class 'int'>

In [52]:
#convert_data.loc[1]

#### 检索dataframe

In [62]:
convert_data.iloc[:1]

风暴潮组  海浪组           海冰组    
             主班   主班  副班  警报班   主班  主班
0                                     
2017-12-01  刘秋兴  韦锋余  徐瑞  王娟娟  唐茂宁  刘琼

In [63]:
convert_data.iloc[0]

风暴潮组  主班     刘秋兴
海浪组   主班     韦锋余
      副班      徐瑞
      警报班    王娟娟
海冰组   主班     唐茂宁
      主班      刘琼
Name: 2017-12-01 00:00:00, dtype: object

In [65]:
convert_data.loc[datetime.datetime(2017,12,1)]

风暴潮组  主班     刘秋兴
海浪组   主班     韦锋余
      副班      徐瑞
      警报班    王娟娟
海冰组   主班     唐茂宁
      主班      刘琼
Name: 2017-12-01 00:00:00, dtype: object

In [69]:
type(convert_data.loc[datetime.datetime(2017,12,1)])

pandas.core.series.Series

In [70]:
convert_data.loc[datetime.datetime(2017,12,1)].index

MultiIndex(levels=[['海冰组', '海浪组', '风暴潮组'], ['主班', '副班', '警报班']],
           labels=[[2, 1, 1, 1, 0, 0], [0, 0, 1, 2, 0, 0]])

#### (2) 选取层次化索引的对象中的数据

In [78]:
convert_data.loc[datetime.datetime(2017,12,1)]['海浪组']

主班     韦锋余
副班      徐瑞
警报班    王娟娟
Name: 2017-12-01 00:00:00, dtype: object

In [79]:
convert_data.loc[datetime.datetime(2017,12,1)]['海浪组','副班']

'徐瑞'

#### （3）获取某一日的一行数据，并分别获取索引的值

In [80]:
convert_data.loc[datetime.datetime(2017,12,1)]

风暴潮组  主班     刘秋兴
海浪组   主班     韦锋余
      副班      徐瑞
      警报班    王娟娟
海冰组   主班     唐茂宁
      主班      刘琼
Name: 2017-12-01 00:00:00, dtype: object

In [81]:
convert_data.loc[datetime.datetime(2017,12,1)].iloc[0]

'刘秋兴'

In [82]:
convert_data.loc[datetime.datetime(2017,12,1)].index

MultiIndex(levels=[['海冰组', '海浪组', '风暴潮组'], ['主班', '副班', '警报班']],
           labels=[[2, 1, 1, 1, 0, 0], [0, 0, 1, 2, 0, 0]])

In [84]:
convert_data.loc[datetime.datetime(2017,12,1)][:1]

风暴潮组  主班    刘秋兴
Name: 2017-12-01 00:00:00, dtype: object

In [85]:
convert_data.loc[datetime.datetime(2017,12,1)][:1].index

MultiIndex(levels=[['海冰组', '海浪组', '风暴潮组'], ['主班', '副班', '警报班']],
           labels=[[2], [0]])

In [89]:
# 使用Unstack进行重塑
convert_df=convert_data.loc[datetime.datetime(2017,12,1)][:1].unstack()
convert_df

,主班
风暴潮组,刘秋兴


In [90]:
convert_df.index

Index(['风暴潮组'], dtype='object')

In [91]:
convert_df.columns

Index(['主班'], dtype='object')